In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
from ansys.mapdl.core import launch_mapdl
from ansys.mapdl import reader as pymapdl_reader

In [ ]:
# Launch MAPDL
mapdl = launch_mapdl()

# Geometric parameter of triangle
n_cell_list = [4, 2, 10, 6, 8]
wall_thickness_ratio_list = [0.1, 0.15, 0.2, 0.35, 0.5]
external_wall_list = [False, True]

for n_cell in n_cell_list:
    for wall_thickness_ratio in wall_thickness_ratio_list:
        for external_wall in external_wall_list:

In [ ]:
design_width = 60/1000
design_height = 25/1000
total_width = 144/1000
# density = 0.5

wall_thickness = wall_thickness_ratio*design_height/n_cell
external_wall_thickness = wall_thickness

In [ ]:


# Data structure to store results
results = []

# Pre-processing
mapdl.clear()
mapdl.prep7()
mapdl.mp("EX", 1, 200e9)  # Young's modulus
mapdl.mp("PRXY", 1, 0.3)  # Poisson's ratio
mapdl.rectng(0, 1, 0, 1)  # Create a rectangular area
mapdl.et(1, "PLANE183")
mapdl.esize(0.1)
mapdl.amesh("ALL")
mapdl.d(1, "ALL")  # Constrain one side

# Perform iterations
for i, force in enumerate(force_values):
    mapdl.f(5, "FX", force)  # Apply force
    mapdl.run("/SOLU")
    mapdl.solve()
    mapdl.finish()
    
    # Post-process results
    mapdl.post1()
    nodal_displacements = mapdl.post_processing.nodal_displacement("X")  # X displacement
    max_displacement = np.max(nodal_displacements)
    von_mises_stress = mapdl.post_processing.nodal_stress("von_mises")
    max_stress = np.max(von_mises_stress)
    
    # Store results
    results.append({
        "Iteration": i + 1,
        "Force": force,
        "Max Displacement": max_displacement,
        "Max Von Mises Stress": max_stress
    })

# Convert results to DataFrame for review
results_df = pd.DataFrame(results)
print(results_df)